In [1]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime
import time
import smtplib

In [ ]:


# Function to check price on Amazon
def check_price():
    URL = 'https://www.amazon.com/dp/B07N12FMPX'
    
    # Headers to mimic a real browser visit
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept-Language": "en-US,en;q=0.9",
    }
    
    response = requests.get(URL, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract product title
        title = soup.find("span", id="productTitle")
        product_name = title.get_text(strip=True) if title else "Title not found"
        
        # Extract product price
        price_element = soup.find("span",  class_="a-offscreen")
        price = price_element.get_text(strip=True)[1:] if price_element else "Price not found"
        
        # Extract product rating
        rating_element = soup.find("span", class_="a-icon-alt")
        product_rating = rating_element.get_text(strip=True) if rating_element else "Rating not found"
        
        # Get today's date
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        
        # Save data into a CSV file
        csv_filename = "AmazonWebScraping.csv"
        header = ['Time','Title', 'Price', 'Rating']
        data = [timestamp, product_name, price , product_rating]
        
        with open(csv_filename, 'a+', newline='', encoding='UTF8') as f:
            writer = csv.writer(f)
            writer.writerow(data)
        
        #If price drops below $15, send an email notification
        if price != "Price not found" and float(price) < 14.99:
            send_mail()
    else:
        print("Failed to retrieve the webpage. Status code:", response.status_code)

# Function to send an email notification when price drops
def send_mail():
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login('example@gmail.com', '**********')  # Replace with your actual email and app password
    
    subject = "The Shirt you want is below $15! Now is your chance to buy!"
    body = "Alex, This is the moment we have been waiting for. Now is your chance to pick up the shirt of your dreams. Don't mess it up! Link here: https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ"
    
    msg = f"Subject: {subject}\n\n{body}"
    
    server.sendmail(
        'example@gmail.com',  # Replace with your email
        'example@gmail.com',  # Replace with recipient's email
        msg
    )
    
    server.quit()
    print("Email has been sent!")

# Run check_price every 24 hours
while True:
    check_price()
    time.sleep(3600)  # Wait for a day before checking again
